In [ ]:
# pip install -U autogen-agentchat autogen-ext[openai,web-surfer]
# playwright install
#%pip install python.dotenv

## Example using the ROUND ROBING GROUPCHAT team with a USER PROXY AGENT and a MULTIMODAL WEBSURFER

In [ ]:

import asyncio
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from dotenv import load_dotenv

from settings import llm_config
load_dotenv()


In [ ]:
model_client = AzureOpenAIChatCompletionClient.load_component(llm_config)

## This requires USER to provide feedback as to what to do next

In [ ]:
async def main() -> None:
    # The web surfer will open a Chromium browser window to perform web browsing tasks.
    web_surfer = MultimodalWebSurfer("web_surfer", model_client, headless=False, animate_actions=True)
    # The user proxy agent is used to get user input after each step of the web surfer.
    # NOTE: you can skip input by pressing Enter.
    user_proxy = UserProxyAgent("user_proxy")
    # The termination condition is set to end the conversation when the user types 'exit'.
    termination = TextMentionTermination("exit", sources=["user_proxy"])
    # Web surfer and user proxy take turns in a round-robin fashion.
    team = RoundRobinGroupChat([web_surfer, user_proxy], termination_condition=termination)
    try:
        # Start the team and wait for it to terminate.
        await Console(team.run_stream(task="find information about the latest iPhone"))
    finally:
        await web_surfer.close()
        await model_client.close()

await main()

## This DOES NOT require user feedback as to what to do next, the orchestrator does that

In [19]:
async def main() -> None:

    surfer = MultimodalWebSurfer(
        "MultimodalWebSurfer",
        model_client=model_client,
        downloads_folder="./downs",
        debug_dir="./debug",
        headless = False,
        to_resize_viewport=True,
        description="A web surfing assistant that can browse and interact with web pages.",
        start_page="https://www.bing.com",  # Optional: Initial page
        animate_actions=True,
        browser_data_dir="./browser_data",
    )
    
    team = MagenticOneGroupChat([surfer], model_client=model_client, max_turns=3)
    # Define a team
    # team = RoundRobinGroupChat([surfer], max_turns=3)
    #await Console(team.run_stream(task="Navigate to the AutoGen readme on GitHub."))
    #await Console(team.run_stream(task="Summarize latest updates from Accenture newsroowm."))
    #await Console(team.run_stream(task="Summarize latest news from venture beat all things in AI."))
    #await Console(team.run_stream(task="Find information about the latest iPhone."))
    await Console(team.run_stream(task="what NHL games are happening today, and tell me about the teams playing"))
    await surfer.close()


if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
await main()

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:158: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:171: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- user ----------
what NHL games are happening today, and tell me about the teams playing
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

what NHL games are happening today, and tell me about the teams playing


To answer this request we have assembled the following team:

MultimodalWebSurfer: A web surfing assistant that can browse and interact with web pages.


Here is an initial fact sheet to consider:

### 1. GIVEN OR VERIFIED FACTS
No explicit facts or figures are provided in the request itself. The only information given is the request to identify NHL games happening "today" and to provide information about the teams playing.

### 2. FACTS TO LOOK UP
- The schedule of NHL games for "today" (specific date inferred from the present day).
- Information about the teams playing in those games (current standings, player stats, team performance history, notable rivalries, etc.).
These details can typically be found on author

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:612: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultimodalWebSurfer ----------
I typed 'NHL schedule today' into the browser search bar.

The web browser is open to the page [NHL schedule today - Search](https://www.bing.com/search?q=NHL+schedule+today&FORM=QBLH).
The viewport shows 31% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

Skip to content
NHL schedule today
FrançaisMobileAll
Search
Shopping
Images
Videos
Maps
News
More
Tools
About 893,000 resultsNHL
2024-25 Season
GAMES
NEWS
STANDINGS
Today
Game 2Canadiens
Capitals
7:00 PM
Today
Game 3Stars
Avalanche
9:30 PM
Today
Game 2Oilers
Kings
10:00 PM
All times are in EDT
Feedback
See more games
NHL
https://www.nhl.com › schedule
Schedule | Apr 21, 2025 | NHL.comFind out the date, time, TV network, and video highlights of the NHL games on Apr 2, 2025. NHL.com is the official website of the NHL with news, scores, standings, and more.
Vancouver CanucksThe official calendar for the Vancouver Canucks including ticket i

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:298: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- MultimodalWebSurfer ----------
I clicked 'Schedule | Apr 21, 2025 | NHL.com'.

The web browser is open to the page [Schedule | Apr 23, 2025 | NHL.com](https://www.nhl.com/schedule).
The viewport shows 100% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

Playoffs
News
Video
Scores
Schedule
Stats
Players
Fantasy
NHL Unites
Teams
Shop
Tickets
EN
NHL Events
Local Broadcast Guide
How to Watch & Stream
NHL Events
Local Broadcast Guide
How to Watch & Stream
More
Schedule
Apr 23 - 29
4/23
Wed
3 Games
4/24
Thu
4 Games
4/25
Fri
3 Games
4/26
Sat
4 Games
4/27
Sun
4 Games
4/28
Mon
2 Games
4/29
Tue
4 Games
Odds by
Hide
Wednesday, April 23
Matchup
(Moneyline)
Time
Broadcasters
Canadiens+150
@
Capitals-182
Round 1
, 
Gm 2
 | 
WSH leads 1-0
7:00 PM EDT
Series
Tickets
Gamecenter
Close
We and our third-party partners may use cookies and similar technologies to enhance site navigation, analyze site usage, save your preferences, enable per

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:951: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------
Please look up team-specific statistics and recent performance for the Canadiens, Capitals, Stars, Avalanche, Oilers, and Kings on NHL.com or another authoritative platform. Summarize each team's context, key players, and current playoff conditions.
---------- MultimodalWebSurfer ----------
I typed 'Canadiens team stats playoff 2025' into the browser search bar.

The web browser is open to the page [Canadiens team stats playoff 2025 - Search](https://www.bing.com/search?q=Canadiens+team+stats+playoff+2025&FORM=QBLH).
The viewport shows 22% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

Skip to content
Canadiens team stats playoff 2025
FrançaisMobileAll
Search
Videos
Images
Maps
News
Shopping
More
Tools
About 888,000 resultsMontreal 
Canadiens stats, scoring leaders and demographics breakdown in the 
2025 NHL 
playoffs. Player nationality and age distributions, draft rounds of players

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:440: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
